## Photometry on MIRI images

In [ ]:
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS, FITSFixedWarning
from astropy.nddata import Cutout2D
from astropy.table import Table
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import glob
import warnings
from scipy.ndimage import zoom

warnings.simplefilter("ignore", category=FITSFixedWarning)


Read in the FITS file from Amir

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)

apr_a = table['Apr_A']
apr_b = table['Apr_B']
x_centre = table['Apr_Xcenter']
y_centre = table['Apr_Ycenter']
theta = table['Apr_Theta']
ids = table['ID']

print(table)

